## Import Libraries

In [17]:
from bs4 import BeautifulSoup
import bs4
import requests
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium_stealth import stealth

## Start Webdriver (Bot)

In [7]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
#options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get("https://bot.sannysoft.com/")

## Innitialize Scrapping

In [8]:
job_list = ['Information-Technology']

In [9]:
resume_links = pd.DataFrame()
category = []
link = []
last_updated_dates = []

## Get link for scrapping

In [10]:
for job in job_list:
    JOB = job.lower()
    for i in range(1,2):   # Jika ingin menambahkan jumlah page untuk di scrape, adjust rangenya
        PAGE = str(i)
        URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
        driver.get(URL)
        aTagsInLi = driver.find_elements(By.CSS_SELECTOR, 'a.sc-1dzblrg-0')
        for a in aTagsInLi:
            category.append(JOB)
            link.append(a.get_attribute('href'))
            last_updated_dates.append(a.find_elements(By.CLASS_NAME, 'last-updated')[0].text)

In [11]:
resume_links["Category"] = category
resume_links["link"] = link
resume_links['last_updated'] = last_updated_dates

In [12]:
resume_links.head()

,Category,link,last_updated
0,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On August 11, 2022"
1,information-technology,https://www.livecareer.com/resume-search/r/hea...,"Last Updated On August 28, 2015"
2,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On June 1, 2015"
3,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On January 16, 2018"
4,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On October 2, 2015"


In [13]:
import hashlib
def id(x):
    return int(hashlib.md5(x.encode('utf-8')).hexdigest(), 16)

resume_links["id"] = resume_links["link"].apply(id)
resume_links.Category.value_counts()

Category
information-technology    10
Name: count, dtype: int64

## Get data from URL

In [14]:
resume_links["Resume"] = ""
resume_links["Raw_html"] = ""

In [18]:
for i in range(resume_links.shape[0]):
    url = resume_links.link[i]
    driver.get(url)
#    print(driver.get(url))
#     time.sleep(0.5)                  #Untuk DELAY
    x = driver.page_source
    x = x.replace(">","> ")
    soup = bs4.BeautifulSoup(x, 'html.parser')
    div = soup.find("div", {"class": "document"})
    resume_links.Raw_html[i] = div
    try:
        print('##############################################')
        resume_links.Resume[i] = div.text
    
    except:
#         ADD EXCEPTION IF REQUIRED
        pass

##############################################
##############################################
##############################################
##############################################
##############################################
##############################################
##############################################
##############################################
##############################################
##############################################


In [19]:
resume_links.head()

,Category,link,last_updated,id,Resume,Raw_html
0,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On August 11, 2022",333384008044094492599850273825632969038,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
1,information-technology,https://www.livecareer.com/resume-search/r/hea...,"Last Updated On August 28, 2015",213083561351819246707939832289245731177,JC J S S JS ...,"[ , [ , [ , <div> <div class=""monogram dynamic..."
2,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On June 1, 2015",57098340265376532323341018871583143087,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
3,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On January 16, 2018",89622349158409716960668547833836774872,JC J S S JS ...,"[ , [ , [ , <div> <div class=""monogram dynamic..."
4,information-technology,https://www.livecareer.com/resume-search/r/inf...,"Last Updated On October 2, 2015",73408001750395583825917262865904570618,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."


In [25]:
df = pd.DataFrame(resume_links)
df = df.drop(['link'], axis=1)

In [26]:
df.dropna(inplace=True)
df.isnull().sum()

Category        0
last_updated    0
id              0
Resume          0
Raw_html        0
dtype: int64

In [28]:
df['id'] = [str(x)[::5] for x in df['id']]

In [29]:
df

,Category,last_updated,id,Resume,Raw_html
0,information-technology,"Last Updated On August 11, 2022",34449769,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
1,information-technology,"Last Updated On August 28, 2015",23527321,JC J S S JS ...,"[ , [ , [ , <div> <div class=""monogram dynamic..."
2,information-technology,"Last Updated On June 1, 2015",53533880,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
3,information-technology,"Last Updated On January 16, 2018",83816738,JC J S S JS ...,"[ , [ , [ , <div> <div class=""monogram dynamic..."
4,information-technology,"Last Updated On October 2, 2015",70089206,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
5,information-technology,"Last Updated On April 17, 2015",28126340,Jessica Claire ...,"[ , [ , [ , <div class=""section notdraggable s..."
6,information-technology,"Last Updated On February 16, 2021",13405733,"Jessica Claire , ...","[ , [ , [ , <div> <div class=""name""> <span id=..."
7,information-technology,"Last Updated On July 27, 2015",27515255,JC J S S JS ...,"[ , [ , [ , <div> <div class=""monogram dynamic..."
8,information-technology,"Last Updated On November 20, 2017",16533554,Jessica Claire Mont...,"[ , [ , [ , <div> <div class=""name""> <span id=..."
9,information-technology,"Last Updated On May 5, 2016",20001721,Jessica Claire ...,"[ , [ , [ , <div class=""section notdraggable s..."


In [30]:
df = df[['id', 'Resume', 'Raw_html', 'Category']]
df.columns = ['ID', 'Resume_str', 'Resume_html', 'Category']
df.Category  = [x.upper() for x in df.Category]

In [ ]:
resume_links.to_csv("Dataset.csv",index=False)

## Convert to PDF

In [ ]:
import pandas as pd
import pdfkit 
import os

In [ ]:
labels = df.Category.unique()
parent_dir = "./new/"

In [ ]:
import os

for label in labels:
    temp = df[df.Category == label]
    temp.dropna(inplace=True)
    temp.reset_index(inplace=True, drop=True)
    path = os.path.join(parent_dir, label)
    mode = 0o666
    if not os.path.exists(path): 
        os.makedirs(path, mode)
    for i in range(temp.shape[0]):
        pdfkit.from_string(temp.Resume_html[i], os.path.join(path, temp.ID[i] + ".pdf"), configuration=pdfkit.configuration(wkhtmltopdf=r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'))
